<a href="https://colab.research.google.com/github/MeenaCherukuri/AIML-Internship/blob/main/12_6_25_AIML_Internship_Project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q librosa soundfile scikit-learn kaggle resampy

import os
import numpy as np
import pandas as pd
import librosa
import soundfile as sf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pickle

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"meenakshicherukuri","key":"38595503e3e6bb9898f73d4e90e52878"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d uwrfkaggler/ravdess-emotional-speech-audio
!unzip -qq ravdess-emotional-speech-audio.zip -d ravdess_data


Dataset URL: https://www.kaggle.com/datasets/uwrfkaggler/ravdess-emotional-speech-audio
License(s): CC-BY-NC-SA-4.0
 91% 392M/429M [00:00<00:00, 436MB/s]
100% 429M/429M [00:01<00:00, 399MB/s]


In [ ]:
# Redundant extract_features function definition removed

In [ ]:
emotion_map = {
    '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
    '05': 'angry', '06': 'fearful', '07': 'disgust', '08': 'surprised'
}


In [ ]:
import librosa
import numpy as np
import soundfile as sf # Import soundfile
import resampy # Explicitly import resampy

def extract_features(file_path):
    print(f"Attempting to extract features for {file_path}") # Debugging line
    try:
        # Use soundfile to load audio
        audio, sample_rate = sf.read(file_path)
        print(f"Successfully loaded audio with soundfile for {file_path}") # Debugging line

        # Resample if necessary (e.g., to a common sample rate like 22050 Hz)
        target_sample_rate = 22050
        if sample_rate != target_sample_rate:
            print(f"Resampling audio from {sample_rate} Hz to {target_sample_rate} Hz for {file_path}") # Debugging line
            audio = resampy.resample(audio, sample_rate, target_sample_rate)
            sample_rate = target_sample_rate
            print(f"Successfully resampled audio for {file_path}") # Debugging line


        # Ensure audio is mono if it's stereo
        if len(audio.shape) > 1:
            audio = librosa.to_mono(audio)


        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)
        print(f"Successfully extracted features for {file_path}") # Debugging line
        return np.hstack([mfccs, chroma, mel])
    except Exception as e:
        print(f"Error extracting features for {file_path}: {e}") # Debugging line
        return None # Return None to indicate failure

In [ ]:
!pip install resampy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 19.9 MB/s eta 0:00:00


In [ ]:
!pip install librosa soundfile scikit-learn resampy kaggle


In [ ]:
import glob

# Define the directory containing the audio files
data_dir = 'ravdess_data/Actor_*'

X = []
y = []

# Iterate over each actor directory
for actor_dir in glob.glob(data_dir):
    # Iterate over each audio file in the actor directory
    for file_path in glob.glob(os.path.join(actor_dir, '*.wav')):
        try:
            # Extract features from the audio file
            features = extract_features(file_path)
            # Append features and label only if feature extraction was successful
            if features is not None:
                X.append(features)

                # Extract the emotion label from the filename
                # The filename format is e.g., '03-01-01-01-02-01-12.wav'
                # The emotion is the third digit
                emotion_code = os.path.basename(file_path).split('-')[2]
                emotion = emotion_map.get(emotion_code)
                if emotion:
                    y.append(emotion)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 1148
Testing samples: 287


In [ ]:
!pip install resampy

In [ ]:
import glob
import os
import numpy as np
from sklearn.model_selection import train_test_split

# Define the directory containing the audio files
data_dir = 'ravdess_data/Actor_*'

X = []
y = []

# Ensure resampy is available immediately before feature extraction
!pip install resampy

# Iterate over each actor directory
for actor_dir in glob.glob(data_dir):
    # Iterate over each audio file in the actor directory
    for file_path in glob.glob(os.path.join(actor_dir, '*.wav')):
        try:
            # Extract features from the audio file
            features = extract_features(file_path)
            # Append features and label only if feature extraction was successful
            if features is not None:
                X.append(features)

                # Extract the emotion label from the filename
                # The filename format is e.g., '03-01-01-01-02-01-12.wav'
                # The emotion is the third digit
                emotion_code = os.path.basename(file_path).split('-')[2]
                emotion = emotion_map.get(emotion_code)
                if emotion:
                    y.append(emotion)

        except Exception as e:
            print(f"Error processing {file_path}: {e}")

# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

print("Shape of X before splitting:", X.shape)
print("Shape of y before splitting:", y.shape)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

Streaming output truncated to the last 5000 lines.
Successfully resampled audio for ravdess_data/Actor_24/03-01-07-02-01-01-24.wav
Successfully extracted features for ravdess_data/Actor_24/03-01-07-02-01-01-24.wav
Attempting to extract features for ravdess_data/Actor_24/03-01-05-02-02-01-24.wav
Successfully loaded audio with soundfile for ravdess_data/Actor_24/03-01-05-02-02-01-24.wav
Resampling audio from 48000 Hz to 22050 Hz for ravdess_data/Actor_24/03-01-05-02-02-01-24.wav
Successfully resampled audio for ravdess_data/Actor_24/03-01-05-02-02-01-24.wav
Successfully extracted features for ravdess_data/Actor_24/03-01-05-02-02-01-24.wav
Attempting to extract features for ravdess_data/Actor_24/03-01-08-01-01-01-24.wav
Successfully loaded audio with soundfile for ravdess_data/Actor_24/03-01-08-01-01-01-24.wav
Resampling audio from 48000 Hz to 22050 Hz for ravdess_data/Actor_24/03-01-08-01-01-01-24.wav
Successfully resampled audio for ravdess_data/Actor_24/03-01-08-01-01-01-24.wav
Success

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))


Training samples: 1148
Testing samples: 287


In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

print("✅ Model training complete.")


✅ Model training complete.


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test)

print("🎯 Accuracy:", round(accuracy_score(y_test, y_pred) * 100, 2), "%")
print("\n📊 Classification Report:\n")
print(classification_report(y_test, y_pred))


🎯 Accuracy: 52.26 %

📊 Classification Report:

              precision    recall  f1-score   support

       angry       0.91      0.54      0.68        39
        calm       0.50      0.74      0.60        38
     disgust       0.42      0.53      0.47        38
     fearful       0.56      0.39      0.46        38
       happy       0.39      0.45      0.41        38
     neutral       0.58      0.37      0.45        19
         sad       0.58      0.54      0.56        39
   surprised       0.51      0.55      0.53        38

    accuracy                           0.52       287
   macro avg       0.56      0.51      0.52       287
weighted avg       0.56      0.52      0.52       287



In [ ]:
import pickle

with open("emotion_model.pkl", "wb") as f:
    pickle.dump(model, f)

print("💾 Model saved as emotion_model.pkl")


💾 Model saved as emotion_model.pkl


In [ ]:
# Reuse your feature extractor function
import librosa
import numpy as np
import soundfile as sf # Import soundfile
import resampy # Explicitly import resampy

def extract_features_for_prediction(file_path):
    print(f"Attempting to extract features for {file_path}") # Debugging line
    try:
        # Use soundfile to load audio
        audio, sample_rate = sf.read(file_path)
        print(f"Successfully loaded audio with soundfile for {file_path}") # Debugging line

        # Resample if necessary (e.g., to a common sample rate like 22050 Hz)
        target_sample_rate = 22050
        if sample_rate != target_sample_rate:
            print(f"Resampling audio from {sample_rate} Hz to {target_sample_rate} Hz for {file_path}") # Debugging line
            audio = resampy.resample(audio, sample_rate, target_sample_rate)
            sample_rate = target_sample_rate
            print(f"Successfully resampled audio for {file_path}") # Debugging line

        # Ensure audio is mono if it's stereo
        if len(audio.shape) > 1:
            audio = librosa.to_mono(audio)


        mfccs = np.mean(librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40).T, axis=0)
        chroma = np.mean(librosa.feature.chroma_stft(y=audio, sr=sample_rate).T, axis=0)
        mel = np.mean(librosa.feature.melspectrogram(y=audio, sr=sample_rate).T, axis=0)
        print(f"Successfully extracted features for {file_path}") # Debugging line
        return np.hstack([mfccs, chroma, mel])
    except Exception as e:
        print(f"Error extracting features for {file_path}: {e}") # Debugging line
        return None # Return None to indicate failure

# Upload your own test audio file
from google.colab import files
uploaded = files.upload()  # Upload a .wav file here

# Load the model
import pickle
model = pickle.load(open("emotion_model.pkl", "rb"))

# Predict
import os
for fname in uploaded.keys():
    features = extract_features_for_prediction(fname)
    if features is not None:
        features = features.reshape(1, -1)
        prediction = model.predict(features)
        print(f"Predicted Emotion for {fname}: {prediction[0]}")
    else:
        print(f"Could not extract features for {fname}, skipping prediction.")

Saving 03-01-02-01-01-01-11.wav to 03-01-02-01-01-01-11.wav
Attempting to extract features for 03-01-02-01-01-01-11.wav
Successfully loaded audio with soundfile for 03-01-02-01-01-01-11.wav
Resampling audio from 48000 Hz to 22050 Hz for 03-01-02-01-01-01-11.wav
Successfully resampled audio for 03-01-02-01-01-01-11.wav
Successfully extracted features for 03-01-02-01-01-01-11.wav
Predicted Emotion for 03-01-02-01-01-01-11.wav: calm
